In [69]:
import numpy as np
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Rescaling
from keras import backend as K
from keras.utils.np_utils import to_categorical
from keras.applications.vgg16 import preprocess_input
from load_images import create_target_encoding
import os
import tensorflow as tf
import matplotlib.pyplot as plt


In [4]:
base_model = keras.applications.vgg16.VGG16(weights='imagenet', input_shape=(224, 244, 3), include_top=False)

58889256/58889256 [==============================] - 27s 0us/step


In [5]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 244, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 244, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 244, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 122, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 122, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 122, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 61, 128)       0     

In [6]:
for layers in base_model.layers[:]: 
    layers.trainable = False 

In [7]:
images_path = './data/'
labels = os.listdir(images_path)

In [8]:
labels

['stuffed_toy', 'small_box', 'shoe', 'cap', 'phone']

In [59]:
def load_image(base_path):
    """it loads all the image into X and the classes in y """
    X_list = []
    y_list = []
    classes = os.listdir(base_path)
    for class_ in classes:
        files = os.listdir(base_path+class_)
        #print(files)
        for file in files:
            if file != ".DS_Store":
                pic = keras.preprocessing.image.load_img(path=base_path+class_+'/'+f'{file}',target_size=(224,224))
                numpy_image = np.array(pic)
                processed_image = preprocess_input(numpy_image)
                X_list.append(processed_image)
                y_list.append(class_)
        
    X = np.array(X_list)
    y = np.array(y_list)
    
    return X, y, classes

In [60]:
X,y,classes= load_image(images_path)

In [61]:
X.shape, y.shape

((560, 224, 224, 3), (560,))

In [62]:
classes

['stuffed_toy', 'small_box', 'shoe', 'cap', 'phone']

In [64]:
y[:10]

array(['stuffed_toy', 'stuffed_toy', 'stuffed_toy', 'stuffed_toy',
       'stuffed_toy', 'stuffed_toy', 'stuffed_toy', 'stuffed_toy',
       'stuffed_toy', 'stuffed_toy'], dtype='<U11')

In [67]:
y_num = create_target_encoding(y)

{'cap': 0, 'phone': 1, 'shoe': 2, 'small_box': 3, 'stuffed_toy': 4}


In [70]:
y_true = y_num.copy()

In [71]:
y_num = to_categorical(y_num)

In [72]:
X_train = np.asarray(X)
y_train = np.asarray(y_num)

In [73]:
classes

['stuffed_toy', 'small_box', 'shoe', 'cap', 'phone']

In [82]:
num_classes = len(classes)
K.clear_session()
model = keras.models.Sequential([
                    base_model,

                    keras.layers.Flatten(),
    
                    keras.layers.Dense(units=30, activation=keras.activations.relu, name='fc1'),
                    
                    keras.layers.Dropout(0.5),

                    keras.layers.Dense(units=15, activation=keras.activations.relu, name='fc2'),
                    
                    keras.layers.Dense(units=num_classes, activation='softmax', name='otput')
                  ])

In [83]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy,
              metrics=['accuracy'])

In [84]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 fc1 (Dense)                 (None, 30)                752670    
                                                                 
 dropout (Dropout)           (None, 30)                0         
                                                                 
 fc2 (Dense)                 (None, 15)                465       
                                                                 
 otput (Dense)               (None, 5)                 80        
                                                                 
Total params: 15,467,903
Trainable params: 753,215
Non-t

In [85]:
history = model.fit(X_train,y_train, validation_split=0.2, batch_size=50, epochs=15)

Epoch 1/15


ValueError: in user code:

    File "/Users/abhi/miniconda3/envs/tensorflow/lib/python3.9/site-packages/keras/engine/training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "/Users/abhi/miniconda3/envs/tensorflow/lib/python3.9/site-packages/keras/engine/training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/abhi/miniconda3/envs/tensorflow/lib/python3.9/site-packages/keras/engine/training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "/Users/abhi/miniconda3/envs/tensorflow/lib/python3.9/site-packages/keras/engine/training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "/Users/abhi/miniconda3/envs/tensorflow/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/abhi/miniconda3/envs/tensorflow/lib/p